In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Water_FINAL.csv')

C:\Users\isaac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3,4,5,6,9,11,13,15,17,21,22,23,24,27,28,31,32,33,34,35,36,37,40,41,42,43,44,45,46,47,48,49,50,55,56,57,58,59,60,61,62,63,64,65,66) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
data['Stream Flow'].unique()

array([nan, 'Normal', 'Dry (Negligible)', 'High', 'Low', 'high', 'normal',
       'low', 'Negligible'], dtype=object)

In [28]:
benthic = pd.read_excel('CMC_Benthic_Data.xlsx')

In [31]:
benthic.columns

Index(['EVENT_ID', 'SAMPLE_NUMBER', 'STATION_ID', 'ICPRB_BIOREGION_ID',
       'STRAHLER_STREAM_ORDER', 'G_Method', 'SAMPLE_DATE', 'MONTH', 'Latitude',
       'Longitude', 'FINAL_ID', 'REPORTING_VALUE', 'TSN_FINAL', 'TAXON_LEVEL',
       'PHYLUM', 'SUBPHYLUM', 'CLASS', 'SUBCLASS', 'ORDER', 'SUBORDER',
       'FAMILY', 'SUBFAMILY', 'TRIBE', 'GENUS', 'SPECIES', 'ASPT', 'BIBI_TV',
       'BIBI_FFG', 'BIBI_HABIT'],
      dtype='object')

In [38]:
benthic['PHYLUM'].unique()

array(['PLATYHELMINTHES', 'ANNELIDA', 'MOLLUSCA', 'ARTHROPODA',
       'UNIDENTIFIED', 'INSECTA', 'MALACOSTRACA', 'GASTROPODA', 0,
       'BIVALVIA', 'OLIGOCHAETA', 'ENTOGNATHA', nan, 'TREPAXONEMATA',
       'HIRUDINEA', 'HYDROZOA', 'ENOPLA'], dtype=object)

In [33]:
benthic.apply(lambda x: x.isnull().sum())

EVENT_ID                      0
SAMPLE_NUMBER                 0
STATION_ID                    0
ICPRB_BIOREGION_ID            0
STRAHLER_STREAM_ORDER    122582
G_Method                      0
SAMPLE_DATE                   0
MONTH                         0
Latitude                      0
Longitude                     0
FINAL_ID                     63
REPORTING_VALUE              63
TSN_FINAL                    63
TAXON_LEVEL                  63
PHYLUM                       63
SUBPHYLUM                    63
CLASS                        63
SUBCLASS                     63
ORDER                        63
SUBORDER                     63
FAMILY                       63
SUBFAMILY                    63
TRIBE                        63
GENUS                        63
SPECIES                      63
ASPT                         63
BIBI_TV                      63
BIBI_FFG                  81229
BIBI_HABIT                81229
dtype: int64

In [18]:
benthic.columns[0:50]

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Active Construction',
       'Agency', 'Algae Color', 'Algae Located', 'Aquatic Veg/Decaying Matter',
       'Area 1 Sampled', 'Area 2 Sampled', 'Area 3 Sampled', 'Area 4 Sampled',
       'Barriers To Fish Movement', 'Benthic Classification Code',
       'Benthic Classification Name', 'BioMethod', 'Bottom Type',
       'Channel Width', 'Collection Time (Net 1)', 'Collection Time (Net 2)',
       'Collection Time (Net 3)', 'Collection Time (Net 4)', 'Comments',
       'Count', 'CountyCity', 'Cropland', 'Database', 'Date', 'DateTime',
       'Define Other Bank Composition', 'Define Other Land Use',
       'Define Other Organism', 'Describe the Amount and Type of Litter',
       'Discharge Pipes', 'EcoRegionLevel3', 'EcoRegionLevel3Description',
       'EventId', 'FIPS', 'Fields', 'Fish Water Quality Indicators', 'Forest',
       'GMethod', 'GroupName', 'HUC12', 'HUC12Description', 'HUC8',
       'HUC8Description', 'Housing Developme

In [22]:
benthic['Comments'].unique()

array([nan])

## Reading HUC12 Data

In [96]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon, LineString
import fiona
from datetime import datetime
#import geoplot as gplt
#import geoplot.crs as gcrs

In [2]:
layers = fiona.listlayers('wbdhu12_a_us_september2019.gdb')
# Read into a geodataframe
huc12 = gpd.read_file('wbdhu12_a_us_september2019.gdb', layer = 'WBDHU12')

In [3]:
# Convert to lat long
huc12.crs = {'init': 'epsg:2283'}

In [4]:
# Filter for the rows that contain the CB states
bay_states = ['NY', 'PA', 'VA', 'MD', 'WV', 'DE', 'DC']
keep = []
for i in bay_states:
    for x in huc12['STATES'].unique():
        if i in x:
            keep.append(x)
        else:
            pass
keephuc = huc12[huc12['STATES'].isin(keep)]

### Read in County Boundaries

In [5]:
counties = gpd.read_file('tl_2017_us_county.shp')

In [6]:
fips = pd.read_csv('us-state-ansi-fips.csv')

In [7]:
# get the state for each county from the FIPs codes
fips['fips'] = ['0' + str(i) if len(str(i)) == 1 else str(i) for i in fips[' st']]
fipsdict = dict(zip(fips['fips'], fips[' stusps'].map(lambda x: x[1:])))
counties['state'] = counties['STATEFP'].map(fipsdict)
counties['fullfips'] = counties['STATEFP'] + counties['COUNTYFP']

keep_counties = counties[counties['state'].isin(bay_states)]

### Read in Water and Benthic Data

In [8]:
water = pd.read_csv('Water_FINAL.csv')
benthic = pd.read_csv('CBP_Benthic.csv')

C:\Users\isaac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,5,6,9,11,13,15,17,21,22,23,24,27,28,31,32,33,34,35,36,37,40,41,42,43,44,45,46,47,48,49,50,55,56,57,58,59,60,61,62,63,64,65,66) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# Getting the geospatial points
water['Point'] = [Point(x, y) for x, y in zip(water['Longitude'], water['Latitude'])]
benthic['Point'] = [Point(x, y) for x, y in zip(benthic['Longitude'], benthic['Latitude'])]

In [10]:
# Get unique points
water_collections = water[['Longitude', 'Latitude']].drop_duplicates()
benthic_collections = benthic[['Longitude', 'Latitude']].drop_duplicates()

allpoints = pd.concat([water_collections, benthic_collections], axis = 0, sort = True)
allpoints.drop_duplicates(inplace = True)
allpoints['Point'] = [Point(x, y) for x, y in zip(allpoints['Longitude'], allpoints['Latitude'])]

In [11]:
# Getting the HUC12 codes for the data

hucs = []
names = []

for i in allpoints['Point']:
    huc = ""
    name = ""
    for x, y, z in zip(keephuc['NAME'], keephuc['HUC12'], keephuc['geometry']):
        if z.contains(i) == True:
            name = x
            huc = y
            break
        else:
            pass
    
    hucs.append(huc)
    names.append(name)

In [12]:
allpoints['HUC12'] = hucs
allpoints['HUCNAME'] = names

In [13]:
# Get the FIPS codes for each point
fips = []
states = []
counties = []
for i in allpoints['Point']:
    
    fip = ""
    state = ""
    county = ""
    for w, x, y, z in zip(keep_counties['fullfips'], keep_counties['state'], keep_counties['NAMELSAD'], keep_counties['geometry']):
            
        if z.contains(i) == True:
            fip = w
            state = x
            county = y
            break
        else:
            pass
            
    fips.append(fip)
    states.append(state)
    counties.append(county)
    
allpoints['FIPS'] = fips
allpoints['STATE'] = states
allpoints['COUNTY'] = counties

In [14]:
allpoints = allpoints.dropna()

In [82]:
allpoints.loc[allpoints['HUC12'] == '020600060202', :]

,Latitude,Longitude,Point,HUC12,HUCNAME,FIPS,STATE,COUNTY
1492798,39.16775,-76.85125,POINT (-76.85124999999999 39.16775),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
4936,39.12900,-76.82860,POINT (-76.82859999999999 39.129),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
5486,39.23770,-76.80790,POINT (-76.8079 39.2377),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
5493,39.15690,-76.91330,POINT (-76.91330000000001 39.1569),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
5500,39.14780,-76.88360,POINT (-76.8836 39.1478),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
...,...,...,...,...,...,...,...,...
107837,39.20115,-76.87915,POINT (-76.87915 39.20115),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
107845,39.20857,-76.86419,POINT (-76.86419000000002 39.20857),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
110993,39.24070,-76.85011,POINT (-76.85011 39.2407),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County
112946,39.21833,-76.81882,POINT (-76.81882 39.21833),020600060202,Dorsey Run-Little Patuxent River,24027,MD,Howard County


In [45]:
allpoints.groupby('HUC12')['STATE'].value_counts().sort_values(ascending = False)

HUC12         STATE
020600060202  MD       129
020600031201  MD        80
020600031102  MD        61
020700080802  MD        59
020600040203  MD        59
                      ... 
020700100603  VA         1
020700100601  VA         1
020700100503  VA         1
020700100307  DC         1
020403030303  MD         1
Name: STATE, Length: 1610, dtype: int64

### Wrangling the Water Data to make sense more

https://www.chesapeakemonitoringcoop.org/wp-content/uploads/2020/07/Data-Dictionary_June-2020.pdf

In [89]:
water['HUC12_'].value_counts()

20600010000    376906
20801010000    286757
20700111001     53982
20801070204     42291
20600040302     35934
                ...  
20502050101         4
20501020702         4
20501020801         4
20502050505         4
20501050203         2
Name: HUC12_, Length: 759, dtype: int64

In [91]:
water.groupby('HUC12_')['coordinates'].nunique().sort_values(ascending = False)

HUC12_
20600040203    59
20600010000    52
20700100204    40
20600040302    31
20802050607    26
               ..
20503050502     1
20503050503     1
20700110904     1
20503050802     1
20403030303     1
Name: coordinates, Length: 759, dtype: int64

In [107]:
water_subset = water[water['HUC12_'] == 20600010000]

In [94]:
water_subset.to_csv('water_subset.csv')

In [109]:
# Change the date
water_subset['Date'] = water_subset['Date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

C:\Users\isaac\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [121]:
water_subset.loc[]

Date
1     1199
2      900
3     1033
4     2266
5     2231
6     3912
7     2968
8     4076
9     2175
10    1814
11    1455
12    1257
Name: HUC12, dtype: int64

In [131]:
water_subset.groupby('Parameter')['Date'].count()

Parameter
AT.3            6
CHL.1         196
CHLA        15030
CO.5          200
DO          66146
DO.2          200
DO.3          200
DO.5           30
NH4F        14777
NO3F        15334
PH          65941
PH.4          200
PO4F        14770
SA.3          200
SA.9           30
SALINITY    66340
SECCHI       6674
TN          14397
TP          14473
TSS         15118
WC.2           23
WT.3          200
WT.4           30
WTEMP       66391
Name: Date, dtype: int64

In [132]:
water_subset.loc[water_subset['Parameter'] == 'TN', 'ParameterName_CBP']

280151     TOTAL NITROGEN
280152     TOTAL NITROGEN
280153     TOTAL NITROGEN
280200     TOTAL NITROGEN
280201     TOTAL NITROGEN
                ...      
2186399    TOTAL NITROGEN
2186400    TOTAL NITROGEN
2186401    TOTAL NITROGEN
2186402    TOTAL NITROGEN
2186454    TOTAL NITROGEN
Name: ParameterName_CBP, Length: 14397, dtype: object

## Plotting Using Mapbox and plotly

In [15]:
import plotly.graph_objects as go
mapbox_access_token = "pk.eyJ1IjoidGhhbXN1cHBwIiwiYSI6ImNrN3Z4eTk2cTA3M2czbG5udDBtM29ubGIifQ.3UvulsJUb0FSLnAOkJiRiA"

In [17]:
# Set the coordinates as a tuple so they can be group-by
water['coordinates'] = water['Point'].apply(lambda point: (point.x, point.y))

There are 2544 unique coordinates, but only 912 unique stations (some coordinates are not stations?)
1622 unique StationCodes and StationNames - but they are 

In [18]:
water.groupby('coordinates')['Point'].count().sort_values(ascending = False)

coordinates
(-76.29215, 38.3187)     28318
(-76.39945, 38.82593)    28240
(-76.22787, 38.13705)    27817
(-76.34565, 38.41457)    27767
(-76.35967, 38.99596)    26170
                         ...  
(-76.0045, 42.1804)          2
(-77.1025, 40.2597)          1
(-79.25378, 38.20305)        1
(-79.29465, 38.27945)        1
(-77.6392, 41.83788)         1
Name: Point, Length: 2544, dtype: int64

In [161]:
water_subset['Year'] = water_subset['Date'].apply(lambda x: x.year)
water_subset['Month'] = water_subset['Date'].apply(lambda x: x.month)
water_subset['Day'] = water_subset['Date'].apply(lambda x: x.day)


C:\Users\isaac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\isaac\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\isaac\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [135]:
water_site_counts = water_subset.groupby('coordinates')['Point'].agg(['count', 'first'])
water_site_counts['Lat'] = water_site_counts['first'].apply(lambda x: x.y)
water_site_counts['Long'] = water_site_counts['first'].apply(lambda x: x.x)

In [136]:
# Initial Mapbox plot of all the points with data points

fig = go.Figure(go.Scattermapbox(
        lat = water_site_counts['Lat'],
        lon = water_site_counts['Long'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size = 5,
            color = water_site_counts['count']
        ),
        text= water_site_counts['count'],
    ))

fig.update_layout(
    height = 800,
    width = 1000,
    margin = {'l': 50, 'r': 50, 't': 0, 'b': 0},
    hovermode='closest',
    showlegend = True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=40,
            lon=-75
        ),
        pitch=0,
        zoom=7
    )
)

fig.show()

In [165]:
a = pd.read_csv('water_subset.csv')

C:\Users\isaac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning:

Columns (6,10,14,16,23,28,32,35,38,43,46,47,49,63,67) have mixed types.Specify dtype option on import or set low_memory=False.



In [166]:
counts = a.groupby('coordinates')['coordinates'].agg(['count', 'first'])

In [150]:
counts['Lat'] = counts['first'].apply(lambda x: float(x[1:-1].split(', ')[1]))
counts['Long'] = counts['first'].apply(lambda x: float(x[1:-1].split(', ')[0]))

In [169]:
year = 2015
month = 12
parameter = 'TN'

In [173]:
param_summary = a.loc[(a['Year'] == year) & (a['Month'] == month) & (a['Parameter'] == parameter), :].groupby('coordinates')[['MeasureValue', 'Longitude', 'Latitude']].mean()

,MeasureValue,Longitude,Latitude
coordinates,,,
"(-76.02599000000002, 39.44149)",1.441550,-76.02599,39.44149
"(-76.17579, 39.348729999999996)",1.125540,-76.17579,39.34873
"(-76.22787, 38.13705)",0.451980,-76.22787,38.13705
"(-76.2405, 39.2495)",0.873775,-76.24050,39.24950
"(-76.29215, 38.3187)",0.392475,-76.29215,38.31870
"(-76.30631, 39.16369)",0.827000,-76.30631,39.16369
"(-76.34565, 38.41457)",0.433825,-76.34565,38.41457
"(-76.35967, 38.99596)",0.585020,-76.35967,38.99596
"(-76.37574000000002, 38.32522)",0.534333,-76.37574,38.32522
